# Understanding ```sympyfy```: An Analytical Momentum-Space Hamiltonian Expresser.

In this notebook, we shall learn the intricacies that go behind the ```sympyfy``` function. But first, let us dive into the motivation behind writing such a function.

The ```sympyfy``` function makes great use of the tight-binding (TB) model. In a nutshell, the TB model is used in Condensed Matter Physics to study the properties of the band structures of different kinds of conductors, semiconductors, and insulators. In particular, we care about how electrons move and interact withing the crystal lattice site framework by considering how they interact with other electrons within nearby lattices. To analyze these interactions, the wavefunctions of the involved electrons should capture their locality. Therefore, we need to choose a set of wavefunctions that can help us characterize the behavior of electrons within the local landscape while neglecting far-off electron interactions. Luckily, there exist a set of localized wavefunctions that can help us do so - Wannier functions. The scope of Wannier functions will be limited in this tutorial. Using these wavefunctions, we can characterize the behavior of electrons across nearby lattice sites as the wave functions of the electrons involved will overlap which will lead to the hopping of electrons from one orbital to another (inter-atomic orbital hopping). These orbital hoppings can be within the same unit cell or across different unit cells and is the main phenomenon that is captured by ```sympyfy```.

The TB model is useful when applied to materials whose electrons are weakly correlated. Like many other models, the TB model has its own Hamiltonian equation. This equation has 2 significant terms, one of which is the hopping term which describes the hopping of electrons between local neighboring sites. This is the term that ```sympyfy``` captures. The TB model captures valuable insights on the electronic properties of materials.
